In [1]:
import json
import time
import itertools
import subprocess

In [2]:
with open("./autotuning.json", "rb") as f:
    config = json.load(f)

In [3]:
template_traversal = \
"""traversal {tname} {{
{tbody}
}}"""
template_case = \
"""    case {cname} {{
        {cbody}
    }}
"""

In [4]:
def get_case_generator(arg_case, arg_nminor):
    all_perms = itertools.permutations(arg_case["major"]+[None for _ in range(arg_nminor)])
    for p in all_perms:
        minor_combs = itertools.combinations(arg_case["minor"], r=arg_nminor)
        for q in minor_combs:
            newp = []
            ind = 0
            for x in p:
                if x is None:
                    newp.append(q[ind])
                    ind += 1
                else:
                    newp.append(x)
                newp.append("??;")
            yield tuple(["??;"]+newp)

In [5]:
def get_traversal_generator(arg_benchmark, arg_nminors):
    case_list = list(arg_benchmark["cases"].keys())
    case_generators = []
    for i in range(len(case_list)):
        dcase = case_list[i]
        tmp_case = get_case_generator(arg_benchmark["cases"][dcase], arg_nminors[i])
        case_generators.append(tmp_case)

    all_cases = itertools.product(*case_generators)
    for prod in all_cases:
        all_str_cases = []
        for i in range(len(prod)):
            str_case = template_case.format(
                cname=case_list[i],
                cbody="\n        ".join(prod[i]),
            )
            all_str_cases.append(str_case)
        str_traversal = template_traversal.format(
            tname=arg_benchmark["name"],
            tbody="\n".join(all_str_cases),
        )
        # print("======")
        # print(str_traversal)
        yield str_traversal

In [7]:
benchmark = "meta-binary-tree.grammar"
# benchmark = "meta-fmm.grammar"
# benchmark = "meta-piecewise-exp1.grammar"
# benchmark = "meta-piecewise-exp2.grammar"
# benchmark = "meta-piecewise-exp3.grammar"
# benchmark = "meta-render.grammar"
nminors = [0 for _ in range(len(config[benchmark]["cases"].keys()))]

# benchmark = "meta-ast.grammar"
# nminors = [0, 0, 1, 0]

tt = get_traversal_generator(config[benchmark], nminors)

with open("./benchmarks/grafter/{}".format(benchmark), "r") as f:
    strb = f.read()
tmp_cnt = -1
for z in tt:
    tmp_cnt += 1
    print("# synthesizing traversal {}...".format(tmp_cnt))
    # instantiate 
#     strz = strb.replace("<auto traversal>", z)
#     print(strz)
    print(z)
    input("PRESS FOR NEXT")
    # put to temporary file
#     fpath = "./tmp/{}-{}".format(str(time.time()), benchmark)
#     with open(fpath, "w") as f:
#         f.write(strz)
        
#     # run hecate
#     p = subprocess.run(
#         ["racket", "./run.rkt", "--root", config[benchmark]["root"], config[benchmark]["name"], fpath], 
#         stdout=subprocess.PIPE, encoding="utf-8"
#     )
#     if p.returncode==0 and "SAT" in p.stdout:
#         print("  # succceed".format(tmp_cnt))
#         print("  # symbolic traversal is:\n{}".format(z))
#         print("  # output is:\n{}".format(p.stdout))
#         break
#     else:
#         print("  # fail".format(tmp_cnt))

# synthesizing traversal 0...
traversal fuse {
    case CRoot {
        ??;
        recur root;
        ??;
    }

    case CNode {
        ??;
        iterate l { ??; recur l; ??; }
        ??;
        iterate r { ??; recur r; ??; }
        ??;
    }

}


PRESS FOR NEXT 


# synthesizing traversal 1...
traversal fuse {
    case CRoot {
        ??;
        recur root;
        ??;
    }

    case CNode {
        ??;
        iterate r { ??; recur r; ??; }
        ??;
        iterate l { ??; recur l; ??; }
        ??;
    }

}


PRESS FOR NEXT 


#### old traversal template

In [121]:
template_traversal = \
"""traversal {tname} {{
{tbody}
}}"""
template_case = \
"""    case {cname} {{
        {cbody}
    }}
"""
def generate_traversal(arg_config):
    ret_traversals = []
    
    # first construct shared components
    major_perms = {}
    minor_perms = {}
    case_perms = {}
    stuffing = {}
    for dkey in arg_config["cases"].keys():
        dcase = arg_config["cases"][dkey]
        # first generate all permutations of major components
        major_list = dcase["major"] + ["??;"]
        major_perms[dkey] = list(itertools.permutations(major_list))
        # then generate all permutations of minor components
        minor_list = dcase["minor"] + ["??;"]
        minor_perms[dkey] = list(itertools.permutations(minor_list))
        # then chain all the minor permutations together
        tmp_stuffing = ()
        for p in minor_perms[dkey]:
            tmp_stuffing += p
        stuffing[dkey] = tmp_stuffing
        # then for every case, construct all candidate permutations
        case_perms[dkey] = []
        for p in major_perms[dkey]:
            tmp_p = ()
            for q in p:
                tmp_p += (q,)
                tmp_p += tmp_stuffing
            tmp_p = tmp_stuffing + tmp_p
            case_perms[dkey].append(tmp_p)
    
#     return case_perms
    
    # convert case permutations to string
    str_case_perms = {}
    for dkey in case_perms.keys():
        tt = case_perms[dkey]
        str_case_perms[dkey] = []
        for pp in tt:
            ss = template_case.format(
                cname=dkey,
                cbody="\n        ".join(pp)
            )
            str_case_perms[dkey].append(ss)

#     return str_case_perms

    # print(str_case_perms)
    # get all final permutations
    all_perms = itertools.product(*(str_case_perms.values()))
    str_all_perms = []
    for p in all_perms:
        # print("p is: {}".format(p))
        str_all_perms.append(template_traversal.format(
            tname=arg_config["name"],
            tbody="\n".join(p)
        ))
    
    return str_all_perms

In [116]:
oo = generate_traversal(config["binary-tree.grammar"])